<a href="https://colab.research.google.com/github/pcbzmani/Learning-repository-for-scala-and-pyspark/blob/main/Solutions/Python/pyspark_max.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 60.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=50cb02d6c9b4c9c0659b7b148054763c18118fa4aa5dff0efe9e903b01f4acca
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("datediff")\
        .config('spark.ui.port', '4040')\
        .getOrCreate()

In [3]:
df = spark.read\
        .format('csv')\
        .option('sep','|')\
        .option('inferSchema',True)\
        .option('header',True)\
        .load('/content/pyspark_max.csv')

In [4]:
df.show()

+---------+-----------------+---------+--------+----------+
|     Date|      Description|   Credit|   Debit|   Balance|
+---------+-----------------+---------+--------+----------+
|12-1-2021|           Salary|87,771.00|    0.00| 87,771.00|
|12-1-2021| Previous_mon_bal|16,922.10|    0.00|104,693.10|
|12-1-2021|           Stocks|     0.00|5,000.00| 99,693.10|
|12-2-2021|   Post_office_rd|     0.00|2,500.00| 97,193.10|
|12-3-2021|          EB_bill|     0.00|  480.00| 96,713.10|
|12-3-2021|             Chit|     0.00|5,000.00| 91,713.10|
|12-3-2021|              SIP|     0.00|  500.00| 91,213.10|
|12-3-2021|           Refund| 1,100.00|    0.00| 92,313.10|
|12-4-2021|          dentist|     0.00|  600.00| 91,713.10|
|12-4-2021|       eating_out|     0.00|  900.00| 90,813.10|
|12-4-2021|    hair_medicine|     0.00|1,722.00| 89,091.10|
|12-4-2021|           petrol|     0.00|  400.00| 88,691.10|
|12-4-2021|        vegtables|     0.00|  250.00| 88,441.10|
|12-5-2021|         land_emi|     0.00|4

In [19]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc,col,max

In [20]:
window_spec = Window\
.partitionBy('Date')\
.orderBy(col('Debit').desc())

In [21]:
df = df.withColumn('max_exp_per_day',max(col('Debit')).over(window_spec))

In [22]:
df.show()

+---------+-----------------+---------+--------+----------+---------------+
|     Date|      Description|   Credit|   Debit|   Balance|max_exp_per_day|
+---------+-----------------+---------+--------+----------+---------------+
|12-1-2021|           Stocks|     0.00|5,000.00| 99,693.10|       5,000.00|
|12-1-2021|           Salary|87,771.00|    0.00| 87,771.00|       5,000.00|
|12-1-2021| Previous_mon_bal|16,922.10|    0.00|104,693.10|       5,000.00|
|12-2-2021|   Post_office_rd|     0.00|2,500.00| 97,193.10|       2,500.00|
|12-3-2021|              SIP|     0.00|  500.00| 91,213.10|         500.00|
|12-3-2021|             Chit|     0.00|5,000.00| 91,713.10|         500.00|
|12-3-2021|          EB_bill|     0.00|  480.00| 96,713.10|         500.00|
|12-3-2021|           Refund| 1,100.00|    0.00| 92,313.10|         500.00|
|12-4-2021|       eating_out|     0.00|  900.00| 90,813.10|         900.00|
|12-4-2021|          dentist|     0.00|  600.00| 91,713.10|         900.00|
|12-4-2021| 